In [ ]:
# 데이터 구성:Series, DataFrame
import pandas as pd

import numpy as np

# 데이터 시각화
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

# 데이터 scaling
from sklearn.preprocessing import StandardScaler
# 데이터 분할:train, test
from sklearn.model_selection import train_test_split

# 분류 Decision Tree
from sklearn.tree import DecisionTreeClassifier
# 분류 Random Forest
from sklearn.ensemble import RandomForestClassifier
# 분류 Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier
# 분류 NN (MLPClassifier)
from sklearn.neural_network import MLPClassifier
# 분류 KNN
from sklearn.neighbors import KNeighborsClassifier
# 로지스틱 회귀
from statsmodels.api import Logit
# 최적 모델, 파라미터 탐색
from sklearn.model_selection import GridSearchCV

# 분류모델 평가 함수
from sklearn.metrics import accuracy_score, f1_score 
from sklearn.metrics import confusion_matrix, classification_report

# 모델 성능 평가: Precision, Recall, F1 Score, ROC Curve, AUC
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve, auc

# 샘플링 : Over-sampling 등
from imblearn.over_sampling import SMOTE


# Check the Frequency of Variable
from collections import Counter

# 매번 분류모델 제작하고 비교하는 코드 짜기 번거로워 만든 분류 종합선물세트
    일단 데이터 아래 세번째 형태에 맞게 해서 순차적으로 다 실행하도록. 그러면 모델별 그래프 합쳐져서 최종적으로 다같이 비교할 수 있도록 막대그래프 시각화돼 나온다.

In [ ]:
# 한글 폰트 적용, 차트에서 음수 표시
matplotlib.rc('font', family='NanumGothic')
matplotlib.rc("axes", unicode_minus = False)

In [ ]:
def fun_data_layout(v_data):
    print("Data 구조:", v_data.shape)
    print()
    print("변수 : ", v_data.columns)
    print()
    #print(v_data.head()) <- Text format... 식별 불편
    # v_data.head()  <- 직접 적용 안됨

# df_raw를 독립변수와 종속변수만 있는 셋으로 정리하고, 아래 순차적으로 진행해주면 됨. 결측치는 당연히 미리 제거해둬야 함.

    SCALE은 아래 전부 출하여부 열로 바꿔야하는거 까먹지말고

In [ ]:
# get_dummies: 데이터의 문자형 변수에 대한 더미변수 생성 
df_raw_dummy = pd.get_dummies(df_raw)

# 설명변수, 목표변수 데이터 지정
df_raw_x = df_raw_dummy.drop(["SCALE"], axis = 1, inplace = False)
df_raw_y = df_raw_dummy["SCALE"] 

# train_test_split(X: 설명변수 데이터, Y: 목표변수 데이터, test_size = test 데이터 비율, random_state: 랜덤)
df_train_x, df_test_x, df_train_y, df_test_y = train_test_split(df_raw_x, # 설명변수 데이터
                                                                df_raw_y, # 목표변수 데이터
                                                                test_size = 0.3, # test 데이터의 비율
                                                                random_state = 1234)  # random state

In [ ]:
# get_dummies: 데이터의 문자형 변수에 대한 더미변수 생성 
df_raw_dummy = pd.get_dummies(df_raw)

# 설명변수, 목표변수 데이터 지정
df_raw_x = df_raw_dummy.drop(["SCALE"], axis = 1, inplace = False)
df_raw_y = df_raw_dummy["SCALE"] 

# train_test_split(X: 설명변수 데이터, Y: 목표변수 데이터, test_size = test 데이터 비율, random_state: 랜덤)
df_train_x, df_test_x, df_train_y, df_test_y = train_test_split(df_raw_x, # 설명변수 데이터
                                                                df_raw_y, # 목표변수 데이터
                                                                test_size = 0.3, # test 데이터의 비율
                                                                random_state = 1234)  # random state

## 로지스틱 회귀(1)_unscaled

In [ ]:
# train_test_split(데이터, test_size = test 데이터 비율, random_state: 랜덤)
df_train, df_test = train_test_split(df_raw, # 데이터
                                     test_size = 0.3, # test 데이터의 비율
                                     random_state = 1234)  # random state

print("train data size : {}".format(df_train.shape))
print("test data size : {}".format(df_test.shape))

In [ ]:
# 귀차니스트를 위한 로지스틱 회귀모델 변수랑 + 타자 안치기 용.
# 종속변수 왼쪽빼는거랑 ~ 하는거 잊지 말고.
'+'.join(df_raw.columns)

In [ ]:
log_model = Logit.from_formula("SCALE~ROLLING_TEMP_T5 + FUR_SZ_TEMP + ROLLING_DESCALING""", df_train)

# 적합
log_result = log_model.fit()

# 결과 출력
print(log_result.summary())

In [ ]:
# train 데이터 예측
y_pred_train = log_result.predict(df_train)
# 0과 1의 값을 가진 class로 변환
y_pred_train_class = (y_pred_train > 0.53).astype(int)
print("Confusion Matrix: \n{}".format(confusion_matrix(df_train["SCALE"],y_pred_train_class)),"\n")

# test 데이터 예측
y_pred_test = log_result.predict(df_test)
# 0과 1의 값을 가진 class로 변환
y_pred_test_class = (y_pred_test > 0.53).astype(int)
print("Confusion Matrix: \n{}".format(confusion_matrix(df_test["SCALE"],y_pred_test_class)),"\n")

# 실제 train 데이터와 예측 결과 비교
print("Train 예측/분류 결과")
print("Accuracy: {0:.3f}\n".format(accuracy_score(df_train["SCALE"], y_pred_train_class)))
print("Confusion Matrix: \n{}".format(confusion_matrix(df_train["SCALE"],y_pred_train_class)),"\n")
print(classification_report(df_train["SCALE"], y_pred_train_class, digits=3))

# 실제 train 데이터와 예측 결과 비교
print("Test 예측/분류 결과")
print("Accuracy: {0:.3f}\n".format(accuracy_score(df_test["SCALE"], y_pred_test_class)))
print("Confusion Matrix: \n{}".format(confusion_matrix(df_test["SCALE"],y_pred_test_class)),"\n")
print(classification_report(df_test["SCALE"], y_pred_test_class, digits=3))

In [ ]:
# 0과 1의 값을 가진 class로 변환: 임계값 변경-> 예측 빈도 달라짐
y_pred_test_class = (y_pred_test > 0.6).astype(int)
print("Confusion Matrix: \n{}".format(confusion_matrix(df_test["SCALE"],y_pred_test_class)),"\n")

In [ ]:
# 실제 train 데이터와 예측 결과 비교
print("Train 예측/분류 결과")
print("Accuracy: {0:.3f}\n".format(accuracy_score(df_train["SCALE"], y_pred_train_class)))
print("Confusion Matrix: \n{}".format(confusion_matrix(df_train["SCALE"],y_pred_train_class)),"\n")
print(classification_report(df_train["SCALE"], y_pred_train_class, digits=3))

# 실제 train 데이터와 예측 결과 비교
print("Test 예측/분류 결과")
print("Accuracy: {0:.3f}\n".format(accuracy_score(df_test["SCALE"], y_pred_test_class)))
print("Confusion Matrix: \n{}".format(confusion_matrix(df_test["SCALE"],y_pred_test_class)),"\n")
print(classification_report(df_test["SCALE"], y_pred_test_class, digits=3))

In [ ]:
train_list = []
test_list = []
for i in range(1,10):
    j = i*0.1
    # 0과 1의 값을 가진 class로 변환
    y_pred_train_class = (y_pred_train > j).astype(int)
    y_pred_test_class = (y_pred_test > j).astype(int)
    train_score = accuracy_score(df_train["SCALE"], y_pred_train_class)
    test_score = accuracy_score(df_test["SCALE"], y_pred_test_class)
    train_list.append(train_score)
    test_list.append(test_score)

df = pd.DataFrame({'train': train_list, 'test': test_list})

index = [0.01*i for i in range(1,100)]

# 모델 정확도 그래프 확인
plt.plot(index, train_list, linestyle = "-", label = "Train Accuracy")
plt.plot(index, test_list, linestyle = "--", label = "Test Accuracy")
plt.xlabel("임계값"); plt.ylabel("accuracy")
plt.legend()

In [ ]:
# 설명변수 중요도
df_logistic_coef = pd.DataFrame({"Coef": log_result.params.values[1:]}, index = log_model.exog_names[1:])
df_logistic_coef.plot.barh(y = "Coef")

# 의사결정 나무

In [ ]:
# Over-sampling 설정
sm = SMOTE(sampling_strategy='auto')

# train데이터를 이용한 Over-sampling
x_resampled, y_resampled = sm.fit_resample(df_train_x,df_train_y)

# 결과 확인
print('Over-Sampling 전:\n',df_train_y.value_counts(),"\n")
print('Over-Sampling 후 Train X: {}'.format(x_resampled.shape))
print('Over-Sampling 후 Train Y: {} \n'.format(y_resampled.shape))

print("Over-Sampling 후 '1':{}".format(sum(y_resampled==1)))
print("Over-Sampling 후 '0':{}".format(sum(y_resampled==0)))

In [ ]:
# 목표변수 빈도 확인
print(df_raw.value_counts(["SCALE"]),"\n")
# print(df_raw["BAD"].value_counts(), "\n")
print("불량 비율  ", df_raw.value_counts(df_raw["SCALE"]==1)/len(df_raw))

In [ ]:
# 혹시나 그리드서치 돌릴지도 모르는 너를 위해
para_depth = [depth for depth in range(1, 7)]
para_split = [n_split * 1 for n_split in range(1, 11)]
para_leaf = [n_leaf * 1 for n_leaf in range(1, 21)]

In [ ]:
estimator = DecisionTreeClassifier(random_state=1234)
# 구하고자 하는 parameter와 범위
param_grid = {"max_depth": para_depth, "min_samples_split": para_split,
              "min_samples_leaf": para_leaf}
# 정확도가 높은 최적 parameter 찾기
grid_dt = GridSearchCV(estimator, param_grid, scoring="accuracy", n_jobs = -1)
grid_dt.fit(x_resampled, y_resampled)

print("best estimator model: \n{}".format(grid_dt.best_estimator_))
print("\nbest parameter: \n{}".format(grid_dt.best_params_))
print("\nbest score: \n{}".format(grid_dt.best_score_.round(3)))

In [ ]:
# export_graphviz: 나무 구조 생성 및 저장 
from sklearn.tree import export_graphviz
# graphviz : 나무 구조 시각화  (.dot 확장자 파일 불러오기 등)
import graphviz

In [ ]:
# 변수명 저장
v_feature_name = df_train_x.columns

tree_low = DecisionTreeClassifier(max_depth= 10, min_samples_leaf= 10, min_samples_split= 8, random_state=1234)
tree_low.fit(df_train_x, df_train_y)

# 트리 모델을 tree_low.dot 파일로 저장. (목표변수, 0: Good, 1: Bad)
export_graphviz(tree_low, out_file="tree_low.dot", class_names = ["양품", "불량"], # 목표변수 값이 숫자-> Label 직접 지정
                # class_names = tree_low.classes_,원래 목표변수 값이 법주형 -> 모델 keyword 사용           
                feature_names = v_feature_name, impurity = True, filled = True)
# graphviz를 이용해 트리 모델 시각화
with open("tree_low.dot") as f:
    dot_graph = f.read()
display(graphviz.Source(dot_graph))

In [ ]:
# 이거 하면 현재 폴더에 DT 이미지 파일 저장됨. 그거 공유해주면 됨.
graphviz.Source(dot_graph).render('tree', format='png')

In [ ]:
# 평가
y_pred = grid_dt.predict(df_test_x)
print("Accuracy: {0:.3f}\n".format(grid_dt.score(df_test_x, df_test_y)))
print("Confusion matrix: \n{}".format(confusion_matrix(df_test_y, y_pred)))

# 목표변수의 빈도 불균형 : f1 score로 모델 평가 
print(classification_report(df_test_y, y_pred, digits=3))

In [ ]:
# tree.feature_importances_로 설명변수 중요도 확인 및 테이블로 저장
df_importance = pd.DataFrame()
df_importance["Feature"] = v_feature_name
df_importance["Importance"] = dt_final.feature_importances_

# df_feature_importance의 테이블을 중요도별로 정렬
df_importance.sort_values("Importance", ascending=False, inplace = True)
df_temp = df_importance.round(3).head(6)
df_temp

In [ ]:
# 설명변수 중요도 그래프
# sort_values : 중요도가 높은 변수를 상위에 그림. 
df_temp.sort_values("Importance", ascending=True, inplace = True)
coordinates = range(len(df_temp))
plt.barh(y = coordinates, width = df_temp["Importance"])
plt.yticks(coordinates, df_temp["Feature"])
plt.xlabel("설명변수 중요도")
plt.ylabel("설명변수")

In [ ]:
# roc curve 그리기, label에 auc 표시(소수점 2자리)
plt.plot(fpr, tpr, label= "AUC = %0.5f"% roc_auc)
plt.legend(loc="lower right")
plt.ylabel("TPR"); plt.xlabel("FPR")
plt.show()

# Random Forrest

    그리드서치 최적값 범위는 DT에서 얼추 찾고 그 언저리로 패러미터 범위 설정해줘라.

In [ ]:
# n_estimatos: 트리 수 변경: 1~150
para_n_tree = [n_tree * 10 for n_tree in range(1, 11)]
# max_depth: 최대 깊이 변경
para_depth = [depth for depth in range(2, 8)]
# min_samples_split: 분할하기 위한 노드의 최소 샘플 수 
para_split = [n_split * 2 for n_split in range(1, 21)]
# min_samples_leaf: 잎사귀 수 제한
para_leaf = [n_leaf * 1 for n_leaf in range(2, 6)]

In [ ]:
para_leaf = [n_leaf * 1 for n_leaf in range(2, 6)]

In [ ]:
estimator = RandomForestClassifier(n_estimators=100,random_state=1234)
# 구하고자 하는 parameter와 범위
param_grid = {"max_depth": para_depth,
              "min_samples_leaf": para_leaf,
             'min_samples_split': para_split}
# 정확도가 높은 최적 parameter 찾기
grid_rf = GridSearchCV(estimator, param_grid, scoring="accuracy", n_jobs = -1)
grid_rf.fit(x_resampled, y_resampled)
print("best estimator model: \n{}".format(grid_rf.best_estimator_))
print("\nbest parameter: \n{}".format(grid_rf.best_params_))
print("\nbest score: \n{}".format(grid_rf.best_score_.round(3)))

In [ ]:
# 평가
y_pred = grid_rf.predict(df_test_x)
print("Accuracy: {0:.3f}\n".format(grid_rf.score(df_test_x, df_test_y)))
print("Confusion matrix: \n{}".format(confusion_matrix(df_test_y, y_pred)))

# 목표변수의 빈도 불균형 : f1 score로 모델 평가 
print(classification_report(df_test_y, y_pred, digits=3))

In [ ]:
print(grid_rf.score(df_train_x, df_train_y))
grid_rf.score(df_test_x, df_test_y)

In [ ]:
rf_final = RandomForestClassifier(max_depth=13, min_samples_leaf=2, min_samples_split=6,
                       random_state=1234)

rf_final.fit(x_resampled, y_resampled)

# tree.feature_importances_로 설명변수 중요도 확인 및 테이블로 저장
df_importance = pd.DataFrame()
df_importance["Feature"] = v_feature_name
df_importance["Importance"] = rf_final.feature_importances_

# df_feature_importance의 테이블을 중요도별로 정렬
df_importance.sort_values("Importance", ascending=False, inplace = True)
df_temp = df_importance.round(3).head(6)
df_temp

In [ ]:
# 설명변수 중요도 그래프
# sort_values : 중요도가 높은 변수를 상위에 그림. 
df_temp.sort_values("Importance", ascending=True, inplace = True)
coordinates = range(len(df_temp))
plt.barh(y = coordinates, width = df_temp["Importance"])
plt.yticks(coordinates, df_temp["Feature"])
plt.xlabel("설명변수 중요도")
plt.ylabel("설명변수")

In [ ]:
# roc curve 그리기, label에 auc 표시(소수점 2자리)
plt.plot(fpr, tpr, label= "AUC = %0.5f"% roc_auc)
plt.legend(loc="lower right")
plt.ylabel("TPR"); plt.xlabel("FPR")
plt.show()

# 그래디언트 부스팅

### 아래 패러미터 범위 고려 DT에서 해보고 고민 조금만 해서 정하기

In [ ]:
# learning_rate 학습률 변경: 0.1 기준
para_lr = [lr * 0.01 for lr in range(7, 14)]
# n_estimatos: 트리 수 변경: 110 기준
para_n_tree = [n_tree * 1 for n_tree in range(85, 95)]
# max_depth: 최대 깊이 변경: 3 기준
para_depth = [depth for depth in range(2, 5)]

In [ ]:
estimator = GradientBoostingClassifier(n_estimators=110, max_depth = 3
                                       ,learning_rate = 0.1 , random_state=1234)
# 구하고자 하는 parameter와 범위
param_grid = {"min_samples_leaf": para_leaf, "learning_rate": para_lr}
# 정확도가 높은 최적 parameter 찾기
grid_gb = GridSearchCV(estimator, param_grid, scoring="accuracy", n_jobs = -1)
grid_gb.fit(x_resampled, y_resampled)
print("best estimator model: \n{}".format(grid_gb.best_estimator_))
print("\nbest parameter: \n{}".format(grid_gb.best_params_))
print("\nbest score: \n{}".format(grid_gb.best_score_.round(3)))

In [ ]:
print(grid_gb.score(df_train_x, df_train_y))
print(grid_gb.score(df_test_x, df_test_y))

In [ ]:
gb_final = GradientBoostingClassifier(min_samples_leaf=4, n_estimators=110,
                           random_state=1234)

gb_final.fit(x_resampled, y_resampled)

# tree.feature_importances_로 설명변수 중요도 확인 및 테이블로 저장
df_importance = pd.DataFrame()
df_importance["Feature"] = v_feature_name
df_importance["Importance"] = gb_final.feature_importances_

# df_feature_importance의 테이블을 중요도별로 정렬
df_importance.sort_values("Importance", ascending=False, inplace = True)
df_temp = df_importance.round(3).head(6)
df_temp

In [ ]:
# 설명변수 중요도 그래프
# sort_values : 중요도가 높은 변수를 상위에 그림. 
df_temp.sort_values("Importance", ascending=True, inplace = True)
coordinates = range(len(df_temp))
plt.barh(y = coordinates, width = df_temp["Importance"])
plt.yticks(coordinates, df_temp["Feature"])
plt.xlabel("설명변수 중요도")
plt.ylabel("설명변수")

In [ ]:
# roc curve 그리기, label에 auc 표시(소수점 2자리)
plt.plot(fpr, tpr, label= "AUC = %0.5f"% roc_auc)
plt.legend(loc="lower right")
plt.ylabel("TPR"); plt.xlabel("FPR")
plt.show()

# XG 부스트

In [ ]:
# xgboost 패키지 불러오기 
from xgboost import XGBClassifier

# 변수 중요도 확인: F1 점수 기준
from xgboost import plot_importance

# 분류모델 평가 함수
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score
from sklearn.metrics import precision_score, recall_score, f1_score

# 분류모델 통합 평가: 혼동행렬, 정확도, 정밀도, 재현율, F1, AUC 등
def eval_class_model(y_test, y_pred):
    confusion = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred)
    AUC = roc_auc_score(y_test, y_pred)
    
    print('오차행렬:\n', confusion, '\n')
    print('정확도: {:.4f}'.format(accuracy))
    print('정밀도: {:.4f}'.format(precision))
    print('재현율: {:.4f}'.format(recall))
    print('F1    : {:.4f}'.format(F1))
    print('AUC   : {:.4f}'.format(AUC))

In [ ]:
# sklearn XGBoost 모델의 조기 중단 : fit( )에 파라미터 입력(early_stopping_rounds, eval_metrics, eval_set.)
# max_depth, learning_rate(학습률), 반복횟수 등 지정
# 오류함수의 평가지표:logloss
# 조기중단을 위한 최소 반복횟수는 150

xgb_stop = XGBClassifier(n_estimators = 110, learning_rate = 0.1 , max_depth = 3)

eval_df = [(df_train_x, df_train_y)]

xgb_stop.fit(x_resampled, y_resampled, early_stopping_rounds = 150, 
                eval_metric="logloss", eval_set = eval_df, verbose=True)

In [ ]:
# 조기 중단 예측 및 모델 평가:Train  
xgb_pred_stop = xgb_stop.predict(df_train_x)

print('\n 조기 중단 모델 평가 : Test \n')
eval_class_model(df_train_y, xgb_pred_stop)
print('\n',classification_report(df_train_y, xgb_pred_stop))

In [ ]:
# tree.feature_importances_로 설명변수 중요도 확인 및 테이블로 저장
df_importance = pd.DataFrame()
df_importance["Feature"] = v_feature_name
df_importance["Importance"] = xgb_stop.feature_importances_

# df_feature_importance의 테이블을 중요도별로 정렬
df_importance.sort_values("Importance", ascending=False, inplace = True)
df_temp = df_importance.round(3).head(6)
df_temp

In [ ]:
# 설명변수 중요도 그래프
# sort_values : 중요도가 높은 변수를 상위에 그림. 
df_temp.sort_values("Importance", ascending=True, inplace = True)
coordinates = range(len(df_temp))
plt.barh(y = coordinates, width = df_temp["Importance"])
plt.yticks(coordinates, df_temp["Feature"])
plt.xlabel("설명변수 중요도")
plt.ylabel("설명변수")

# KNN

In [ ]:
estimator = KNeighborsClassifier()

# 구하고자 하는 parameter와 범위
grid_params = {
    'n_neighbors' : list(range(1,10)),
    'weights' : ["uniform", "distance"],
    'metric' : ['euclidean', 'manhattan', 'minkowski']
}
# 정확도가 높은 최적 parameter 찾기
grid_knn = GridSearchCV(estimator, grid_params, scoring="accuracy", n_jobs = -1)
grid_knn.fit(x_resampled, y_resampled)
print("best estimator model: \n{}".format(grid_rf.best_estimator_))
print("\nbest parameter: \n{}".format(grid_rf.best_params_))
print("\nbest score: \n{}".format(grid_rf.best_score_.round(3)))

In [ ]:
print(grid_knn.score(df_train_x, df_train_y))
print(grid_knn.score(df_test_x, df_test_y))

# NN

#### 제발 얘도 범위 고려. 평생소원.

In [ ]:
# activation 변경 
para_function = ["logistic", "tanh", "relu"]
# hidden_layer_sizes: 은닉층 변경 (20~240, by 30)
para_hidden = [1 * hidden for hidden in range(148, 160)]
# solver 변경 
para_solver = ["lbfgs", "sgd", "adam"]

In [ ]:
estimator = MLPClassifier(random_state=1234)
# 구하고자 하는 parameter와 범위
param_grid = {"activation": para_function,
              "hidden_layer_sizes": para_hidden}
# 정확도가 높은 최적 parameter 찾기
grid_nn = GridSearchCV(estimator, param_grid, scoring="accuracy", n_jobs = -1)
grid_nn.fit(x_resampled, y_resampled)
print("best estimator model: \n{}".format(grid_rf.best_estimator_))
print("\nbest parameter: \n{}".format(grid_rf.best_params_))
print("\nbest score: \n{}".format(grid_rf.best_score_.round(3)))

In [ ]:
print(grid_nn.score(df_train_x, df_train_y))
print(grid_nn.score(df_test_x, df_test_y))

# 분석모델 그래프 대통합

In [ ]:
# 분석 모델
model = ["Logistic", "DecisionTree", "RandomForest", "GradientBoosting"
         , "XGBoosting", "K-NearestNEighbor", "NeuralNet"]

# 정확도 저장
train_accuracy = []; test_accuracy = []
# auc score 저장
model_auc = []

# Precision, Recall score 저장
model_precision = []; model_recall = []
# f1 score 저장
model_f1_score = []

## 아래 SCALE 종속변수 컬럼명으로 바꿔라

In [ ]:
# train 정확도
train_accuracy.append(accuracy_score(df_train["SCALE"], y_pred_train_class))
# test 정확도
test_accuracy.append(accuracy_score(df_test["SCALE"], y_pred_test_class))

# 예측값
y_pred = (y_pred_test > 0.53).astype(int)

# roc_curve(실제값, 예측값), fpr = FP/(FP+TN): 거짓 양성 비율, tpr = TP/(TP+FN): 진짜 양성 비율(재현율)
fpr, tpr, thresholds = roc_curve(df_test_y, y_pred)
roc_auc = auc(fpr, tpr)

# auc 저장
model_auc.append(roc_auc)

# precision, recall score
model_precision.append(precision_score(df_test_y, y_pred))
model_recall.append(recall_score(df_test_y, y_pred))

# f1 score
model_f1_score.append(f1_score(df_test_y, y_pred))

In [ ]:
dt_final = DecisionTreeClassifier(max_depth=10
                                           , min_samples_leaf=1
                                           , min_samples_split=8, random_state= 1234)

dt_final.fit(df_train_x, df_train_y)

# train 정확도
train_accuracy.append(dt_final.score(df_train_x, df_train_y))
# test 정확도
test_accuracy.append(dt_final.score(df_test_x, df_test_y))

# 예측값
y_pred = dt_final.predict(df_test_x)

# roc_curve(실제값, 예측값), fpr = FP/(FP+TN): 거짓 양성 비율, tpr = TP/(TP+FN): 진짜 양성 비율(재현율)
fpr, tpr, thresholds = roc_curve(df_test_y, y_pred)
roc_auc = auc(fpr, tpr)

# auc 저장
model_auc.append(roc_auc)

# precision, recall score
model_precision.append(precision_score(df_test_y, y_pred))
model_recall.append(recall_score(df_test_y, y_pred))

# f1 score
model_f1_score.append(f1_score(df_test_y, y_pred))

In [ ]:
# train 정확도
train_accuracy.append(grid_rf.score(df_train_x, df_train_y))
# test 정확도
test_accuracy.append(grid_rf.score(df_test_x, df_test_y))

# 예측값
y_pred = grid_rf.predict(df_test_x)

# roc_curve(실제값, 예측값), fpr = FP/(FP+TN): 거짓 양성 비율, tpr = TP/(TP+FN): 진짜 양성 비율(재현율)
fpr, tpr, thresholds = roc_curve(df_test_y, y_pred)
roc_auc = auc(fpr, tpr)

# auc 저장
model_auc.append(roc_auc)

# precision, recall score
model_precision.append(precision_score(df_test_y, y_pred))
model_recall.append(recall_score(df_test_y, y_pred))

# f1 score
model_f1_score.append(f1_score(df_test_y, y_pred))

In [ ]:
# train 정확도
train_accuracy.append(gb_final.score(df_train_x, df_train_y))
# test 정확도
test_accuracy.append(gb_final.score(df_test_x, df_test_y))

# 예측값
y_pred = gb_final.predict(df_test_x)

# roc_curve(실제값, 예측값), fpr = FP/(FP+TN): 거짓 양성 비율, tpr = TP/(TP+FN): 진짜 양성 비율(재현율)
fpr, tpr, thresholds = roc_curve(df_test_y, y_pred)
roc_auc = auc(fpr, tpr)

# auc 저장
model_auc.append(roc_auc)

# precision, recall score
model_precision.append(precision_score(df_test_y, y_pred))
model_recall.append(recall_score(df_test_y, y_pred))

# f1 score
model_f1_score.append(f1_score(df_test_y, y_pred))

In [ ]:
# train 정확도
train_accuracy.append(xgb_stop.score(df_train_x, df_train_y))
# test 정확도
test_accuracy.append(xgb_stop.score(df_test_x, df_test_y))

# 예측값
y_pred = xgb_stop.predict(df_test_x)

# roc_curve(실제값, 예측값), fpr = FP/(FP+TN): 거짓 양성 비율, tpr = TP/(TP+FN): 진짜 양성 비율(재현율)
fpr, tpr, thresholds = roc_curve(df_test_y, y_pred)
roc_auc = auc(fpr, tpr)

# auc 저장
model_auc.append(roc_auc)

# precision, recall score
model_precision.append(precision_score(df_test_y, y_pred))
model_recall.append(recall_score(df_test_y, y_pred))

# f1 score
model_f1_score.append(f1_score(df_test_y, y_pred))

In [ ]:
# train 정확도
train_accuracy.append(grid_knn.score(df_train_x, df_train_y))
# test 정확도
test_accuracy.append(grid_knn.score(df_test_x, df_test_y))

# 예측값
y_pred = grid_knn.predict(df_test_x)

# roc_curve(실제값, 예측값), fpr = FP/(FP+TN): 거짓 양성 비율, tpr = TP/(TP+FN): 진짜 양성 비율(재현율)
fpr, tpr, thresholds = roc_curve(df_test_y, y_pred)
roc_auc = auc(fpr, tpr)

# auc 저장
model_auc.append(roc_auc)

# precision, recall score
model_precision.append(precision_score(df_test_y, y_pred))
model_recall.append(recall_score(df_test_y, y_pred))

# f1 score
model_f1_score.append(f1_score(df_test_y, y_pred))

In [ ]:
# train 정확도
train_accuracy.append(nn_final.score(df_train_x, df_train_y))
# test 정확도
test_accuracy.append(nn_final.score(df_test_x, df_test_y))

# 예측값
y_pred = kk_final.predict(df_test_x)

# roc_curve(실제값, 예측값), fpr = FP/(FP+TN): 거짓 양성 비율, tpr = TP/(TP+FN): 진짜 양성 비율(재현율)
fpr, tpr, thresholds = roc_curve(df_test_y, y_pred)
roc_auc = auc(fpr, tpr)

# auc 저장
model_auc.append(roc_auc)

# precision, recall score
model_precision.append(precision_score(df_test_y, y_pred))
model_recall.append(recall_score(df_test_y, y_pred))

# f1 score
model_f1_score.append(f1_score(df_test_y, y_pred))

In [ ]:
df_eval = pd.DataFrame(index = model)
df_eval['TrainAccuracy'] = train_accuracy ; df_eval['TestAccuracy'] = test_accuracy
df_eval['AUC'] = model_auc
df_eval['Precision'] = model_precision ; df_eval['Recall'] = model_recall
df_eval['F1Score'] = model_f1_score
df_eval.round(3)

In [ ]:
# 모델별 평가 지표 확인
df_eval.plot.bar(rot = 0, figsize=(10,6))
plt.legend(loc="lower right")
plt.grid(axis = "y")